In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

In [ ]:
# Read the file from the drive
file_path = '/content/drive/My Drive/FinalDataset.tsv'
df = pd.read_csv(file_path, delimiter='\t', error_bad_lines=False)

<ipython-input-5-c4991a1ac648>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, delimiter='\t', error_bad_lines=False)
Skipping line 17089: expected 15 fields, saw 22



In [ ]:
df_clean = df[['star_rating','review_body']]

df_clean = df_clean.sample(frac=0.01)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 9644 to 16104
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   star_rating  199 non-null    int64 
 1   review_body  199 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00


In [ ]:
df_clean['sentiment'] = df_clean['star_rating'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))
df_clean = df_clean.rename(columns={'Reviews': 'review_body'})

In [ ]:
df_clean

,star_rating,review_body,sentiment
9644,5,This was a perfect choice for a large wedding ...,2
5859,5,Easy set up. Works as advertised,2
15585,1,Not even close to stainless steel. Worst quali...,0
6063,4,Boyfriend loved it,2
1386,5,Nice product!,2
...,...,...,...
18556,5,It was just as I expected.,2
8953,5,Thank you!,2
10637,5,Love the cute design for Father's Day,2
11859,5,I bought this for myself. Thanks self!,2


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Define the Dataset
class AmazonReviewDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx], self.labels[idx]

In [ ]:
# Initialize GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained('gpt2-medium', num_labels=3)
model.resize_token_embeddings(len(tokenizer))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warn

In [ ]:
# Tokenize the dataset
inputs = tokenizer(df_clean['review_body'].tolist(), truncation=True, padding=True, max_length=100, return_tensors='pt')
input_ids, attention_masks = inputs['input_ids'], inputs['attention_mask']
sentiments = df_clean['sentiment'].to_numpy()

# Split data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, sentiments, test_size=0.1
)

In [ ]:
train_dataset = AmazonReviewDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = AmazonReviewDataset(val_inputs, val_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for input_ids, attention_masks, labels in train_loader:
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}')

Epoch: 1, Training Loss: 0.9863
Epoch: 2, Training Loss: 0.5815
Epoch: 3, Training Loss: 0.6441


In [ ]:
# Evaluation
model.eval()
val_preds = []
val_true = []

for input_ids, attention_masks, labels in val_loader:
    input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits
        pred = torch.argmax(logits, dim=1).cpu().numpy()
        val_preds.extend(pred)
        val_true.extend(labels.cpu().numpy())

# Function to map original labels to new categories
def map_sentiment(label):
    if label in [1, 2]:  # Assuming original labels are 1, 2, 3, 4, 5
        return 'negative'
    elif label == 3:
        return 'neutral'
    else:
        return 'positive'

# Map the original labels to the new categories
val_true_mapped = [map_sentiment(label) for label in val_true]
val_preds_mapped = [map_sentiment(label) for label in val_preds]

# Classification report
from sklearn.metrics import classification_report
report = classification_report(val_true_mapped, val_preds_mapped)
print(report)

from sklearn.metrics import confusion_matrix

# Compute and print the confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(val_true_mapped, val_preds_mapped, labels=['negative', 'neutral', 'positive']))



              precision    recall  f1-score   support

    negative       0.87      1.00      0.93        13
    positive       1.00      0.71      0.83         7

    accuracy                           0.90        20
   macro avg       0.93      0.86      0.88        20
weighted avg       0.91      0.90      0.90        20


Confusion Matrix:
 [[13  0  0]
 [ 0  0  0]
 [ 2  0  5]]


In [ ]:
# Bi-LSTM
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import GPT2Tokenizer

# Preprocess data
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token


max_length = 200

def encode_reviews(tokenizer, reviews, max_length):
    return tokenizer.batch_encode_plus(reviews, truncation=True, padding='max_length', max_length=max_length)

# Assuming df_clean is your DataFrame
encoded_reviews = encode_reviews(tokenizer, df_clean['review_body'].tolist(), max_length)

input_ids = torch.tensor(encoded_reviews['input_ids'])
sentiments = torch.tensor(df_clean['sentiment'].tolist())

# Define the BiLSTM model
class BiLSTMForSentimentClassification(nn.Module):
    def __init__(self, vocab_size, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 100, padding_idx=pad_idx)
        self.lstm = nn.LSTM(100, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(256*2, 3)

    def forward(self, text):
        embedded = self.embedding(text)
        packed_output, (hidden, cell) = self.lstm(embedded)
        out = self.fc(packed_output[:, -1, :])
        return out

PAD_IDX = tokenizer.pad_token_id
model = BiLSTMForSentimentClassification(len(tokenizer), PAD_IDX).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)

# Splitting data
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, sentiments, test_size=0.1)
train_data = TensorDataset(train_inputs, train_labels)
val_data = TensorDataset(val_inputs, val_labels)

BATCH_SIZE = 16
train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_data, shuffle=False, batch_size=BATCH_SIZE)

# Training loop
for epoch in range(4):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))

from sklearn.metrics import confusion_matrix

# Compute and print the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:\n", confusion_matrix(val_true_mapped, val_preds_mapped, labels=['negative', 'neutral', 'positive']))


Epoch 1, Loss: 0.9931023518244425
Epoch 2, Loss: 0.8152206689119339
Epoch 3, Loss: 0.638461240256826
Epoch 4, Loss: 0.6962716281414032
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         1
           2       0.70      1.00      0.82        14

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20


Confusion Matrix:
 [[13  0  0]
 [ 0  0  0]
 [ 2  0  5]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Multinomial Naive Bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

# Splitting the data
train_texts, val_texts, train_labels_nb, val_labels_nb = train_test_split(df_clean['review_body'], df_clean['sentiment'], test_size=0.1)

model_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())

model_nb.fit(train_texts, train_labels_nb)
predictions = model_nb.predict(val_texts)

print(classification_report(val_labels_nb, predictions))

from sklearn.metrics import confusion_matrix

# Compute and print the confusion matrix
cm = confusion_matrix(val_labels_nb, predictions)
print("\nConfusion Matrix:\n", confusion_matrix(val_true_mapped, val_preds_mapped, labels=['negative', 'neutral', 'positive']))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           2       0.75      1.00      0.86        15

    accuracy                           0.75        20
   macro avg       0.38      0.50      0.43        20
weighted avg       0.56      0.75      0.64        20


Confusion Matrix:
 [[13  0  0]
 [ 0  0  0]
 [ 2  0  5]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
